In [9]:
# Celda para calcular el promedio de la temperatura dentro del cooler.
import pandas as pd
import re

# Nombre del archivo (debe estar en la misma carpeta que el .ipynb)
filename = "20250815_0800-0800.TXT"

# Leer todas las líneas
with open(filename, "r", encoding="utf-8", errors="replace") as f:
    lines = f.readlines()

# Saltar la primera línea (Inicio: ...)
data_lines = lines[1:]

registros = []

for line in data_lines:
    # Extraer valores numéricos de sensores (solo números bien formados tipo 13.50 o 14)
    valores = re.findall(r"S\d+:\s*([\d]+(?:\.\d+)?)", line)
    if valores:
        # Convertir a float
        valores = list(map(float, valores))
        registros.append(valores)

# Crear DataFrame con S1..S19
df = pd.DataFrame(registros, columns=[f"S{i}" for i in range(1, 20)])

# Calcular el promedio general de todas las lecturas y sensores
promedio_total = df.mean().mean()

print(f"Promedio total de temperatura: {promedio_total:.2f} °C")


Promedio total de temperatura: 13.75 °C


In [ ]:
# === Mínimos por sensor y globales (impresión en pantalla) + promedio de 10 mín por sensor ===
# Lee 20250819_0800-0800.TXT, detecta formato (clave:valor o crudo),
# normaliza sensores a S1..SN (según lo que haya), y muestra:
# 0) Promedio de las 10 mín por sensor (idealmente 190 valores si hay 19 sensores)
# 1) 10 más bajas por sensor (con timestamp)
# 2) 10 más bajas globales (con sensor y timestamp)

import re
import pandas as pd
from datetime import datetime

INPUT_FILE = "20250819_0800-0800.TXT"

# Regex para: YYYY-MM-DD,HH:MM:SS,resto
LINE_RE = re.compile(
    r'^(\d{4}-\d{2}-\d{2})\s*,\s*([0-9]{2}:[0-9]{2}:[0-9]{2})\s*,\s*(.*)\s*$'
)

def to_float(x):
    """Convierte string a float o None; maneja '?', vacío y números con punto final."""
    x = (x or "").strip().rstrip(".")
    if x in ("", "?"):
        return None
    try:
        return float(x)
    except ValueError:
        return None

rows = []
seen_sensor_idx = set()
total_lines = 0
valid_lines = 0

with open(INPUT_FILE, "r", encoding="utf-8-sig", errors="replace") as f:
    for raw in f:
        total_lines += 1
        line = raw.strip()
        if not line:
            continue

        m = LINE_RE.match(line)
        if not m:
            continue
        valid_lines += 1

        date_str, time_str, rest = m.group(1), m.group(2), m.group(3)
        # Estructura base
        row = {"Date": date_str, "Time": time_str}

        # Partes separadas por coma
        parts = [p.strip() for p in rest.strip(",").split(",") if p.strip()]

        if parts and ":" in parts[0]:
            # Formato A: clave:valor
            kv = {}
            for item in parts:
                if ":" not in item:
                    continue
                k, v = item.split(":", 1)
                k, v = k.strip(), v.strip()
                kv[k] = v

            # Unidad (opcional)
            if "Unidad" in kv:
                row["Unidad"] = kv["Unidad"]

            # Detecta sensores S1..Sx presentes en la línea
            for k, v in kv.items():
                if k.upper().startswith("S"):
                    # Extrae índice numérico (S1, S2, etc.)
                    try:
                        idx = int(k[1:])
                        seen_sensor_idx.add(idx)
                        row[f"S{idx}"] = to_float(v)
                    except Exception:
                        pass

        else:
            # Formato B: "Unidad, v1, v2, ..., vN"
            if not parts:
                continue
            # La primera suele ser Unidad (C)
            row["Unidad"] = parts[0]
            values = parts[1:]
            # Asigna S1..SN según la cantidad encontrada en esta línea
            for i, v in enumerate(values, start=1):
                seen_sensor_idx.add(i)
                row[f"S{i}"] = to_float(v)

        rows.append(row)

if not rows:
    print("No se encontraron líneas válidas con timestamp.")
else:
    # Construye DataFrame y columnas de sensores vistas
    df = pd.DataFrame(rows)
    # Ordena y asegura todas las columnas S1..SN observadas
    max_idx = max(seen_sensor_idx) if seen_sensor_idx else 0
    sensor_cols = [f"S{i}" for i in range(1, max_idx + 1)]
    for c in sensor_cols:
        if c not in df.columns:
            df[c] = None

    # Timestamp completo
    df["TS"] = pd.to_datetime(df["Date"] + " " + df["Time"], errors="coerce")

    # Conversión a float segura (por si algún sensor quedó como objeto)
    for c in sensor_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # ---------- (0) PROMEDIO de las 10 mín por sensor (idealmente 190 valores) ----------
    all_min_values = []
    used_counts = {}

    for c in sensor_cols:
        s = df[c].dropna()
        if s.empty:
            used_counts[c] = 0
            continue
        # Índices de las 10 más bajas (o menos si no hay tantas)
        idxs = s.nsmallest(10).index
        vals = s.loc[idxs].astype(float).tolist()
        all_min_values.extend(vals)
        used_counts[c] = len(vals)

    expected_total = len(sensor_cols) * 10
    got_total = len(all_min_values)
    avg_10mins = (pd.Series(all_min_values).mean() if all_min_values else float("nan"))

    print(f"Archivo: {INPUT_FILE}")
    print(f"Líneas leídas: {total_lines} | Con timestamp válido: {valid_lines}")
    print(f"Sensores detectados: {len(sensor_cols)} -> {', '.join(sensor_cols)}")
    print(f"Promedio de las 10 mín por sensor (esperado {expected_total}, usados {got_total} valores): "
          f"{avg_10mins:.3f} °C\n")

    # ========== (1) 10 más bajas por sensor ==========
    print("=== 10 temperaturas más bajas POR SENSOR ===")
    for c in sensor_cols:
        s = df[["TS", c]].dropna(subset=[c])
        if s.empty:
            print(f"\n{c}: (sin datos)")
            continue
        idxs = s[c].nsmallest(10).index
        out = s.loc[idxs].rename(columns={c: "Temp_C"}).sort_values("Temp_C")
        print(f"\n{c}:")
        # Imprime TS y temperatura
        for _, r in out.iterrows():
            print(f"  {r['TS']}  ->  {r['Temp_C']:.3f} °C")

    # ========== (2) 10 más bajas GLOBALES ==========
    print("\n=== 10 temperaturas más bajas GLOBALES (todos los sensores) ===")
    # “Derretir” a formato largo: (TS, Sensor, Temp_C)
    long = (
        df[["TS"] + sensor_cols]
        .melt(id_vars=["TS"], var_name="Sensor", value_name="Temp_C")
        .dropna(subset=["Temp_C"])
    )
    if long.empty:
        print("(sin datos)")
    else:
        idxs = long["Temp_C"].nsmallest(10).index
        outg = long.loc[idxs].sort_values("Temp_C")
        for _, r in outg.iterrows():
            print(f"  {r['TS']}  |  {r['Sensor']}  ->  {r['Temp_C']:.3f} °C")


Archivo: 20250819_0800-0800.TXT
Líneas leídas: 1209 | Con timestamp válido: 1208
Sensores detectados: 19 -> S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15, S16, S17, S18, S19
Promedio de las 10 mín por sensor (esperado 190, usados 190 valores): -1.176 °C

=== 10 temperaturas más bajas POR SENSOR ===

S1:
  2025-08-19 15:32:22  ->  -3.060 °C
  2025-08-19 15:33:22  ->  -3.000 °C
  2025-08-19 15:34:22  ->  -3.000 °C
  2025-08-19 15:35:22  ->  -3.000 °C
  2025-08-19 15:36:22  ->  -3.000 °C
  2025-08-19 15:37:22  ->  -3.000 °C
  2025-08-19 15:39:22  ->  -3.000 °C
  2025-08-19 15:40:22  ->  -3.000 °C
  2025-08-19 15:41:22  ->  -3.000 °C
  2025-08-19 15:30:22  ->  -2.940 °C

S2:
  2025-08-19 15:32:22  ->  -0.630 °C
  2025-08-19 15:33:22  ->  -0.630 °C
  2025-08-19 15:34:22  ->  -0.630 °C
  2025-08-19 15:35:22  ->  -0.630 °C
  2025-08-19 15:36:22  ->  -0.630 °C
  2025-08-19 15:37:22  ->  -0.630 °C
  2025-08-19 15:38:22  ->  -0.630 °C
  2025-08-19 15:39:22  ->  -0.630 °C
  202

In [1]:
# Celda para crear el merged_log_R.txt.
"""
Fusiona (merge) una lista de archivos de texto en un único archivo de salida,
con reporte por archivo (encontrado/omitido/líneas escritas) y opción para
saltar la primera línea si es cabecera. Maneja codificación y errores.
"""

# --- User settings ---
files_to_merge = [
    '20250819_0800-0800.TXT',
]

output_filename = "merged_log_R.txt"

# Si tus archivos realmente TIENEN cabecera y quieres omitirla:
SKIP_FIRST_LINE = False     # pon True sólo si estás seguro de que hay cabecera en cada archivo
ENCODING = 'utf-8'          # cambia a 'utf-8-sig' si sospechas BOM

# ---------------------
total_written = 0                 # contador global de líneas escritas al archivo final
file_reports = {}                 # dict para guardar un resumen por archivo procesado

try:
    # open(..., 'w', ...) abre el archivo de salida en modo escritura (crea/trunca).
    # encoding='utf-8' especifica la codificación; errors='replace' reemplaza caracteres inválidos.
    with open(output_filename, 'w', encoding='utf-8', errors='replace') as outfile:
        # Itera sobre la lista de archivos de entrada a fusionar
        for filename in files_to_merge:
            try:
                # open(..., 'r', ...) abre cada archivo de entrada en modo lectura.
                # Se usa ENCODING configurable y errors='replace' para evitar romperse por caracteres raros.
                with open(filename, 'r', encoding=ENCODING, errors='replace') as infile:
                    # readlines() lee TODO el archivo y devuelve una lista de líneas (incluye '\n' si está).
                    lines = infile.readlines()
                    original_len = len(lines)  # número de líneas leídas del archivo original

                    if not lines:
                        # Archivo vacío: informamos y seguimos al siguiente
                        file_reports[filename] = {"found": True, "lines_in": 0, "written": 0, "note": "vacío"}
                        print(f"Warning: '{filename}' está vacío.")
                        continue

                    # Si decidiste omitir la primera línea (cabecera), se puede detectar de forma simple:
                    if SKIP_FIRST_LINE:
                        # strip() elimina espacios/saltos de línea al inicio/fin
                        first = lines[0].strip()
                        # Heurística de cabecera: contiene 'Date' o 'Unidad:' sin comas tipo CSV
                        if ('Date' in first) or ('Unidad' in first and ':' in first and ',' not in first):
                            # lines[1:] crea una sublista desde la segunda línea (omite cabecera)
                            lines = lines[1:]

                    # Asegurar que la ÚLTIMA línea termine en '\n' para que no se "pegue" con la primera del siguiente archivo
                    if lines and not lines[-1].endswith('\n'):
                        # endswith() verifica si la cadena termina con un sufijo dado
                        lines[-1] = lines[-1] + '\n'

                    # writelines(lista) escribe todas las líneas de una vez en el archivo de salida
                    outfile.writelines(lines)

                    written = len(lines)   # cuántas líneas de este archivo se volcaron realmente
                    total_written += written

                    # Guardamos un pequeño reporte por archivo
                    file_reports[filename] = {
                        "found": True,          # el archivo existía
                        "lines_in": original_len,  # líneas leídas originalmente
                        "written": written,        # líneas escritas (luego de omitir cabecera, si aplica)
                        "note": ""                 # campo libre para notas
                    }

                    print(f"OK: '{filename}' -> {written} líneas añadidas (de {original_len}).")

            except FileNotFoundError:
                # Captura cuando el archivo no existe; lo marcamos como omitido
                file_reports[filename] = {"found": False, "lines_in": None, "written": 0, "note": "no encontrado"}
                print(f"Warning: Archivo no encontrado y omitido: '{filename}'")
            except Exception as e:
                # Cualquier otro error de lectura/parsing se registra y se continúa
                file_reports[filename] = {"found": True, "lines_in": None, "written": 0, "note": f"error: {e}"}
                print(f"Error al procesar '{filename}': {e}")

    # Al salir del with, outfile.close() se ejecuta automáticamente (cierre seguro del archivo).
    print(f"\nMerge finalizado. Líneas totales escritas en '{output_filename}': {total_written}")

    # Resumen final por archivo procesado
    print("\n=== Resumen por archivo ===")
    for fn, rep in file_reports.items():
        # print() imprime cada entrada del dict de reporte
        print(f"{fn}: {rep}")

except Exception as e:
    # Manejo de excepción global (por ejemplo, permisos, disco, etc.)
    print(f"Error global durante el merge: {e}")


OK: '20250819_0800-0800.TXT' -> 1209 líneas añadidas (de 1209).

Merge finalizado. Líneas totales escritas en 'merged_log_R.txt': 1209

=== Resumen por archivo ===
20250819_0800-0800.TXT: {'found': True, 'lines_in': 1209, 'written': 1209, 'note': ''}


In [4]:
#Celda para crear merged_testR.csv 1
"""Lee 'merged_log_R.txt' línea a línea, parsea fecha y hora, filtra por rango entre start_datetime y end_datetime, extrae pares clave:valor (S1, S2, Unidad, etc.), arma un DataFrame y guarda 'merged_testR.csv'."""

import pandas as pd  # pd.DataFrame(...) crea tablas en memoria; .to_csv(...) exporta a CSV
from datetime import datetime, time  # datetime.combine(...), datetime.strptime(...), time.min/max definen y parsean fechas/horas

# --- User settings ---
# Componentes de fecha de inicio/fin que puedes ajustar / Start/end date components you can adjust
start_year = 2025
start_month = 8
start_day = 19

end_year = 2025
end_month = 8
end_day = 20

# Nombres de archivo de entrada y salida / Input and output file names
input_filename = 'merged_log_R.txt'
output_filename = "merged_testR.csv"
# ---------------------


# --- Code ---
# datetime(...) construye un objeto fecha-hora con precisión de segundo
# time.min/time.max → 00:00:00 y 23:59:59.999999 respectivamente
# datetime.combine(date, time) fusiona fecha y hora en un único datetime
start_datetime = datetime.combine(datetime(start_year, start_month, start_day), time.min)
end_datetime   = datetime.combine(datetime(end_year, end_month, end_day),   time.max)

# Lista donde acumularemos cada fila válida como diccionario
processed_rows = []

try:
    # with open(..., 'r') abre archivo en modo lectura y asegura cierre automático al salir del bloque
    with open(input_filename, 'r') as f:
        # for line in f itera línea a línea sin cargar todo a memoria
        for line in f:
            line = line.strip()  # str.strip() elimina espacios y saltos de línea al inicio/fin

            # Validación rápida: omite líneas vacías o que no comienzan con dígito (no parecen datos)
            # str.isdigit() comprueba si el primer carácter es un número
            if not line or not line[0].isdigit():
                continue

            try:
                # str.strip(',') elimina coma final si existe; str.split(',') separa en lista por comas
                data_list = line.strip(',').split(',')

                # Construye "YYYY-MM-DD HH:MM:SS" a partir de las dos primeras columnas
                line_datetime_str = f"{data_list[0]} {data_list[1]}"
                # datetime.strptime(cadena, formato) convierte texto a datetime según el patrón dado
                line_datetime = datetime.strptime(line_datetime_str, '%Y-%m-%d %H:%M:%S')

                # Filtro por RANGO de fecha-hora (inclusive)
                # Comparación directa entre objetos datetime es válida en Python
                if start_datetime <= line_datetime <= end_datetime:

                    # Diccionario base con columnas Date y Time
                    row_dict = {
                        'Date': data_list[0],
                        'Time': data_list[1]
                    }

                    # Recorre el resto de ítems con forma "clave:valor"
                    for item in data_list[2:]:
                        if ':' in item:  # Verifica que haya separador clave:valor
                            # str.split(':', 1) divide solo en la primera aparición → evita cortar valores con más ':'
                            key, value = item.split(':', 1)
                            key = key.strip()     # str.strip() limpia espacios accidentales
                            value = value.strip()

                            # Para sensores (p. ej., 'S1','S2'...) convertimos el valor a float
                            # str.startswith('S') detecta claves que inician con 'S'
                            # str.rstrip('.') quita un punto final si viene pegado (p.ej. "19.7.")
                            if key.startswith('S'):
                                row_dict[key] = float(value.rstrip('.'))
                            else:
                                # Otras claves (Unidad, etc.) se guardan como texto
                                row_dict[key] = value

                    # list.append(...) agrega el diccionario de la fila procesada a la lista
                    processed_rows.append(row_dict)

            # Captura errores comunes:
            # ValueError (fallo al convertir tipos o al hacer strptime) e IndexError (acceso fuera de rango en data_list)
            except (ValueError, IndexError) as e:
                # print(...) muestra un aviso pero continúa el bucle
                print(f"Skipping malformed line: {line} - Error: {e}")
                continue

    # Si se recolectó al menos una fila válida:
    if processed_rows:
        # pd.DataFrame(lista_de_dicts) crea un DataFrame con columnas = claves del dict
        final_df = pd.DataFrame(processed_rows)
        # DataFrame.to_csv(ruta, index=False) guarda CSV sin columna de índice
        final_df.to_csv(output_filename, index=False)
        print(f"Processing complete. Data between {start_datetime} and {end_datetime} has been saved to '{output_filename}'.")
    else:
        print("No data was found within the specified date range.")

# Manejo de excepciones a nivel de archivo:
except FileNotFoundError:
    print(f"Error: The input file '{input_filename}' was not found. Please make sure the file is in the same directory as the script.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Processing complete. Data between 2025-08-19 00:00:00 and 2025-08-20 23:59:59.999999 has been saved to 'merged_testR.csv'.


In [1]:
# Celda para crear merged_testR.csv 2  (filtra por hora y minuto ,  fuerza 19 sensores)
"""Lee 'merged_log_R.txt', toma solo líneas válidas entre start_datetime y end_datetime, pero desde 2025-08-14 14:07:36 inclusive.
Acepta dos formatos de línea (clave:valor o valores crudos), normaliza a exactamente 19 sensores (S1..S19),
rellena faltantes con NaN y convierte "?" en NaN. Guarda 'merged_testR.csv' listo para graficar (heatmap)."""

import re
import pandas as pd
from datetime import datetime, time

# --- User settings ---
# Define fechas de inicio/fin, aunque la hora exacta mínima la fija START_EXACT_DATETIME
start_year, start_month, start_day = 2025, 8, 19
end_year,   end_month,   end_day   = 2025, 8, 20

# datetime(...) genera el instante exacto desde el que se empieza a aceptar datos
START_EXACT_DATETIME = datetime(2025, 8, 19, 15, 22, 22)

input_filename  = 'merged_log_R.txt'
output_filename = 'merged_testR.csv'
# ---------------------

# datetime.combine(date, time) une fecha con hora mínima/máxima del día
start_datetime = datetime.combine(datetime(start_year, start_month, start_day), time.min)
end_datetime   = datetime.combine(datetime(end_year,   end_month,   end_day),   time.max)

# re.compile(...) define un patrón regex para validar líneas con timestamp y resto de datos
LINE_RE_KV = re.compile(r'^(\d{4}-\d{2}-\d{2})\s*,\s*([0-9]{2}:[0-9]{2}:[0-9]{2})\s*,\s*(.*)\s*$')

def to_float(x):
    """Convierte string a float o None; maneja '?', vacío y errores."""
    x = (x or '').strip().rstrip('.')
    if x in ('', '?'):
        return None
    try:
        return float(x)
    except ValueError:
        return None

processed_rows = []
total_lines = 0
matched_lines = 0
parsed_rows = 0

# with open(...) abre archivo con codificación utf-8-sig para eliminar BOM si existe
with open(input_filename, 'r', encoding='utf-8-sig', errors='replace') as f:
    for raw in f:
        total_lines += 1
        line = raw.strip()
        if not line:
            continue

        # Aplica regex a la línea
        m = LINE_RE_KV.match(line)
        if not m:
            continue
        matched_lines += 1

        date_str, time_str, rest = m.group(1), m.group(2), m.group(3)
        # datetime.strptime convierte string en datetime
        line_dt = datetime.strptime(f"{date_str} {time_str}", "%Y-%m-%d %H:%M:%S")

        # Filtrado temporal: dentro del rango y mayor/igual a START_EXACT_DATETIME
        if not (start_datetime <= line_dt <= end_datetime):
            continue
        if line_dt < START_EXACT_DATETIME:
            continue

        # Estructura base del registro
        row = {'Date': date_str, 'Time': time_str, 'Unidad': None}

        # parts = lista de valores tras dividir por coma
        parts = [p.strip() for p in rest.strip(',').split(',') if p.strip()]

        # Detecta formato A (clave:valor) o B (valores crudos)
        if parts and ':' in parts[0]:
            # Formato A
            values_dict = {}
            for item in parts:
                if ':' not in item:
                    continue
                k, v = item.split(':', 1)
                k, v = k.strip(), v.strip()
                values_dict[k] = v

            # Extrae unidad si existe
            if 'Unidad' in values_dict:
                row['Unidad'] = values_dict['Unidad']

            # Fuerza S1..S19
            for i in range(1, 20):
                key = f"S{i}"
                row[key] = to_float(values_dict.get(key, None))

        else:
            # Formato B
            if parts:
                row['Unidad'] = parts[0]
                sensor_vals = parts[1:]  # lista de valores numéricos
                # Se toma exactamente 19: se trunca si son más, o se rellena con None si son menos
                sensor_vals = sensor_vals[:19]
                if len(sensor_vals) < 19:
                    sensor_vals = sensor_vals + [None] * (19 - len(sensor_vals))
                for i in range(19):
                    row[f"S{i+1}"] = to_float(sensor_vals[i])
            else:
                continue

        processed_rows.append(row)
        parsed_rows += 1

# Genera DataFrame y asegura columnas ordenadas
if processed_rows:
    df = pd.DataFrame(processed_rows)
    cols_front = ['Date', 'Time', 'Unidad']
    sensors = [f"S{i}" for i in range(1, 20)]
    for c in sensors:
        if c not in df.columns:
            df[c] = None
    df = df[cols_front + sensors]
    df.to_csv(output_filename, index=False)
    print(f"OK: {len(df)} filas guardadas en '{output_filename}'. "
          f"(desde {START_EXACT_DATETIME} con S1..S19)")
else:
    print("No se obtuvieron filas en el rango/desde el instante indicado.")

print(f"Total líneas leídas: {total_lines} | Con timestamp válido: {matched_lines} | Filas agregadas: {parsed_rows}")


OK: 998 filas guardadas en 'merged_testR.csv'. (desde 2025-08-19 15:22:22 con S1..S19)
Total líneas leídas: 1209 | Con timestamp válido: 1208 | Filas agregadas: 998


In [ ]:
# Celda para crear merged_testR.csv 2.1  (filtra por hora y minuto ,  fuerza 19 sensores)
"""Lee 'merged_log_R.txt', toma solo líneas válidas dentro de [start_datetime, end_datetime],
pero además permite fijar límites EXACTOS con START_EXACT_DATETIME y END_EXACT_DATETIME.
Acepta dos formatos de línea (clave:valor o valores crudos), normaliza a exactamente 19 sensores (S1..S19),
rellena faltantes con NaN y convierte "?" en NaN. Guarda 'merged_testR.csv' listo para graficar (heatmap)."""

import re
import pandas as pd
from datetime import datetime, date, time

# --- User settings ---
# Ventana diaria gruesa (por si quieres acotar por días completos)
start_year, start_month, start_day = 2025, 8, 19
end_year,   end_month,   end_day   = 2025, 8, 20

# Límites EXACTOS (dominan sobre la ventana por día):
# - Si START_EXACT_DATETIME no es None, se exige fecha/hora >= START_EXACT_DATETIME (o > si INCLUDE_START=False).
# - Si END_EXACT_DATETIME   no es None, se exige fecha/hora <= END_EXACT_DATETIME   (o < si INCLUDE_END=False).
START_EXACT_DATETIME = datetime(2025, 8, 19, 15, 22, 22)
END_EXACT_DATETIME   = datetime(2025, 8, 20, 7, 59, 22)  # Pon None para desactivar límite exacto final

# Inclusividad de los límites exactos
INCLUDE_START = True  # True => >= START_EXACT_DATETIME; False => > START_EXACT_DATETIME
INCLUDE_END   = True  # True => <= END_EXACT_DATETIME;   False => < END_EXACT_DATETIME

input_filename  = 'merged_log_R.txt'
output_filename = 'merged_testR.csv'
# ---------------------

# Construye ventana diaria “gruesa”
start_datetime = datetime.combine(date(start_year, start_month, start_day), time.min)
end_datetime   = datetime.combine(date(end_year,   end_month,   end_day),   time.max)

LINE_RE_KV = re.compile(r'^(\d{4}-\d{2}-\d{2})\s*,\s*([0-9]{2}:[0-9]{2}:[0-9]{2})\s*,\s*(.*)\s*$')

def to_float(x):
    """Convierte string a float o None; maneja '?', vacío y errores."""
    x = (x or '').strip().rstrip('.')
    if x in ('', '?'):
        return None
    try:
        return float(x)
    except ValueError:
        return None

processed_rows = []
total_lines = 0
matched_lines = 0
parsed_rows = 0

with open(input_filename, 'r', encoding='utf-8-sig', errors='replace') as f:
    for raw in f:
        total_lines += 1
        line = raw.strip()
        if not line:
            continue

        m = LINE_RE_KV.match(line)
        if not m:
            continue
        matched_lines += 1

        date_str, time_str, rest = m.group(1), m.group(2), m.group(3)
        line_dt = datetime.strptime(f"{date_str} {time_str}", "%Y-%m-%d %H:%M:%S")

        # 1) Filtro por ventana diaria gruesa
        if not (start_datetime <= line_dt <= end_datetime):
            continue

        # 2) Filtros EXACTOS (si están definidos)
        if START_EXACT_DATETIME is not None:
            if INCLUDE_START:
                if line_dt < START_EXACT_DATETIME:
                    continue
            else:
                if line_dt <= START_EXACT_DATETIME:
                    continue

        if END_EXACT_DATETIME is not None:
            if INCLUDE_END:
                if line_dt > END_EXACT_DATETIME:
                    continue
            else:
                if line_dt >= END_EXACT_DATETIME:
                    continue

        # Registro base
        row = {'Date': date_str, 'Time': time_str, 'Unidad': None}

        parts = [p.strip() for p in rest.strip(',').split(',') if p.strip()]

        # A) Formato clave:valor
        if parts and ':' in parts[0]:
            values_dict = {}
            for item in parts:
                if ':' not in item:
                    continue
                k, v = item.split(':', 1)
                k, v = k.strip(), v.strip()
                values_dict[k] = v

            if 'Unidad' in values_dict:
                row['Unidad'] = values_dict['Unidad']

            for i in range(1, 20):
                key = f"S{i}"
                row[key] = to_float(values_dict.get(key, None))

        # B) Formato valores crudos
        else:
            if not parts:
                continue
            row['Unidad'] = parts[0]
            sensor_vals = parts[1:]
            sensor_vals = sensor_vals[:19]
            if len(sensor_vals) < 19:
                sensor_vals = sensor_vals + [None] * (19 - len(sensor_vals))
            for i in range(19):
                row[f"S{i+1}"] = to_float(sensor_vals[i])

        processed_rows.append(row)
        parsed_rows += 1

# Export
if processed_rows:
    df = pd.DataFrame(processed_rows)
    cols_front = ['Date', 'Time', 'Unidad']
    sensors = [f"S{i}" for i in range(1, 20)]
    for c in sensors:
        if c not in df.columns:
            df[c] = None
    df = df[cols_front + sensors]
    df.to_csv(output_filename, index=False)

    # Reporte con primer/último timestamp procesado
    first_ts = pd.to_datetime(df['Date'] + ' ' + df['Time']).min()
    last_ts  = pd.to_datetime(df['Date'] + ' ' + df['Time']).max()

    print(f"OK: {len(df)} filas guardadas en '{output_filename}'.")
    print(f"Rango real procesado: {first_ts}  →  {last_ts}")
    if START_EXACT_DATETIME is not None:
        print(f"Inicio exacto {'incluido' if INCLUDE_START else 'excluido'}: {START_EXACT_DATETIME}")
    if END_EXACT_DATETIME is not None:
        print(f"Final exacto {'incluido' if INCLUDE_END else 'excluido'}: {END_EXACT_DATETIME}")
else:
    print("No se obtuvieron filas en el rango/desde los instantes indicados.")

print(f"Total líneas leídas: {total_lines} | Con timestamp válido: {matched_lines} | Filas agregadas: {parsed_rows}")


OK: 6950 filas guardadas en 'merged_testR.csv'.
Rango real procesado: 2025-08-14 12:12:11  →  2025-08-19 07:59:46
Inicio exacto incluido: 2025-08-14 12:12:11
Final exacto incluido: 2025-08-19 07:59:46
Total líneas leídas: 6961 | Con timestamp válido: 6956 | Filas agregadas: 6950


In [ ]:
#Celda para crear merged_testR.csv 3 (es como la 1 pero con limites de hora)
"""Lee 'merged_log_R.txt', filtra filas por un rango de fecha-hora configurable (día completo o intervalo y/min específicos),
extrae pares clave:valor (S1, S2, Unidad, etc.), arma un DataFrame y guarda 'merged_testR.csv'."""

import pandas as pd  # pd.DataFrame(...) crea tablas en memoria; .to_csv(...) exporta a CSV
from datetime import datetime, time  # datetime(...), datetime.combine(...), time.min/max definen y combinan fechas/horas

# --- User settings ---
# Activa un rango de tiempo específico; si es False usa todo el día / Toggle specific time range; if False use the full day
USE_SPECIFIC_TIME_RANGE = True 

# Componentes de fecha de inicio/fin que puedes ajustar / Start/end date components you can adjust
start_year = 2025
start_month = 8
start_day = 14
end_year = 2025
end_month = 8
end_day = 19

# --- Optional: Specific time range ---
# Variables de hora/minuto para el rango específico / Hour/minute variables for the specific range
start_hour = 8   # Hora de inicio (24h) / Start hour (24h)
start_minute = 0 # Minuto de inicio / Start minute
end_hour = 8     # Hora de término / End hour
end_minute = 0   # Minuto de término / End minute
# ------------------------------------

# Nombres de archivo de entrada y salida / Input and output file names
input_filename = 'merged_log_R.txt'
output_filename = "merged_testR.csv"
# ---------------------


# --- Code ---
# Si se usa rango específico, construye datetimes con año/mes/día/hora/minuto / If using specific range, build datetimes with y/m/d/h/min
if USE_SPECIFIC_TIME_RANGE:
    try:
        # datetime(año,mes,día,hora,minuto) crea un objeto datetime con precisión de minuto
        start_datetime = datetime(start_year, start_month, start_day, start_hour, start_minute)
        end_datetime   = datetime(end_year,   end_month,   end_day,   end_hour,   end_minute)
    except NameError:
        # print(...) muestra mensaje en consola; exit() finaliza el proceso
        print("Error: USE_SPECIFIC_TIME_RANGE is True, but the time variables (start_hour, etc.) are not defined.")
        print("Please make sure to set the time variables in the user settings.")
        exit()  # Finaliza el programa / Terminates the program
else:
    # datetime.combine(date, time) combina fecha con la hora mínima/máxima del día
    # time.min/time.max corresponden a 00:00:00 y 23:59:59.999999
    start_datetime = datetime.combine(datetime(start_year, start_month, start_day), time.min)
    end_datetime   = datetime.combine(datetime(end_year,   end_month,   end_day),   time.max)


# Lista para acumular filas que cumplan el filtro / List to collect rows that meet the filter
processed_rows = []  # list.append(x) agregará cada fila procesada

# try/except para manejar errores de archivo u otros / try/except to handle file or other errors
try:
    # with open(ruta,'r') abre archivo en lectura y lo cierra automáticamente al salir del bloque
    with open(input_filename, 'r') as f:
        for line in f:  # Itera línea a línea sin cargar todo a memoria
            line = line.strip()  # str.strip() remueve espacios/saltos al inicio/fin
            # Salta líneas vacías o que no empiezan con dígito (no son datos) / Skip empty or non-data lines
            # str.isdigit() verifica si el primer carácter es un dígito
            if not line or not line[0].isdigit():
                continue

            try:
                # str.strip(',') quita coma final si existe; str.split(',') separa en lista por comas
                data_list = line.strip(',').split(',')

                # Construye "YYYY-MM-DD HH:MM:SS" y parsea con datetime.strptime(cadena,formato)
                line_datetime_str = f"{data_list[0]} {data_list[1]}"
                line_datetime = datetime.strptime(line_datetime_str, '%Y-%m-%d %H:%M:%S')

                # Verifica condición de rango de fecha-hora (inclusive en extremos)
                if start_datetime <= line_datetime <= end_datetime:
                    # Diccionario base por fila / Per-row dictionary
                    row_dict = {
                        'Date': data_list[0],
                        'Time': data_list[1]
                    }

                    # Recorre campos "clave:valor" (unidad, sensores, etc.)
                    for item in data_list[2:]:
                        if ':' in item:  # Verifica separador ':' con operador de pertenencia 'in'
                            # str.split(':',1) divide solo en la primera aparición → evita romper valores con ':'
                            key, value = item.split(':', 1)
                            key = key.strip()      # str.strip() elimina espacios
                            value = value.strip()
                            
                            # Para sensores (claves que empiezan con 'S'): convierte a float
                            # str.startswith('S') detecta prefijo; str.rstrip('.') quita punto final si viene pegado (p.ej. "19.7.")
                            if key.startswith('S'):
                                row_dict[key] = float(value.rstrip('.'))
                            else:
                                # Otras claves se guardan como texto
                                row_dict[key] = value

                    # list.append(...) agrega el diccionario de fila a la colección
                    processed_rows.append(row_dict)

            # ValueError: errores de conversión (float, strptime)
            # IndexError: acceso fuera de rango en data_list
            except (ValueError, IndexError) as e:
                # print(...) informa línea omitida y continúa el bucle
                print(f"Skipping malformed line: {line} - Error: {e}")
                continue

    # Si hubo filas procesadas, crear DataFrame y guardar CSV
    if processed_rows:
        # pd.DataFrame(lista_de_dicts) → columnas a partir de las claves de los dicts
        final_df = pd.DataFrame(processed_rows)
        # DataFrame.to_csv(ruta, index=False) guarda CSV sin columna índice adicional
        final_df.to_csv(output_filename, index=False)
        print(f"Processing complete. Data between {start_datetime} and {end_datetime} has been saved to '{output_filename}'.")
    else:
        print("No data was found within the specified date range.")

# Manejo de errores específicos y genéricos
except FileNotFoundError:
    print(f"Error: The input file '{input_filename}' was not found. Please make sure the file is in the same directory as the script.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Skipping malformed line: 2025-08-14,12:06:31, Unidad: C°,S1: ?, S2: ?, S3: ?, S4: ?, S5: ?, S6: ?, S7: ?, S8: ?, S9: ?, S10: ?, S11: ?, S12: ?, S13: ?, S14: ?, S15: ?, S16: ?, S17: ?, S18: ?, S19: ?, S20: ?. - Error: could not convert string to float: '?'
Skipping malformed line: 2025-08-14,12:07:31, Unidad: C°,S1: ?, S2: ?, S3: ?, S4: ?, S5: ?, S6: ?, S7: ?, S8: ?, S9: ?, S10: ?, S11: ?, S12: ?, S13: ?, S14: ?, S15: ?, S16: ?, S17: ?, S18: ?, S19: ?, S20: ?. - Error: could not convert string to float: '?'
Skipping malformed line: 2025-08-14,12:08:31, Unidad: C°,S1: ?, S2: ?, S3: ?, S4: ?, S5: ?, S6: ?, S7: ?, S8: ?, S9: ?, S10: ?, S11: ?, S12: ?, S13: ?, S14: ?, S15: ?, S16: ?, S17: ?, S18: ?, S19: ?, S20: ?. - Error: could not convert string to float: '?'
Skipping malformed line: 2025-08-14,12:09:31, Unidad: C°,S1: ?, S2: ?, S3: ?, S4: ?, S5: ?, S6: ?, S7: ?, S8: ?, S9: ?, S10: ?, S11: ?, S12: ?, S13: ?, S14: ?, S15: ?, S16: ?, S17: ?, S18: ?, S19: ?, S20: ?. - Error: could not conve

In [2]:
#Celda para crear Datalog_merged_log_R.csv 1
"""Lee 'merged_testR.csv', extrae lecturas de sensores S# mediante regex, arma un DataFrame con columnas ordenadas
(Date, Time, Unit, S1..Sn) y exporta 'Datalog_merged_log_R.csv'. Omite encabezados y líneas mal formadas."""

import pandas as pd  # pd.DataFrame(...) crea tablas en memoria; .to_csv(...) exporta a CSV
import re            # re.findall(...) busca patrones (S#: valor) en texto

# --- User settings ---
# Nombres de archivo de entrada/salida / Input/output file names
input_filename = 'merged_testR.csv'
cleaned_filename = 'cleaned_merged_log_R.csv'  # (reservado para usos posteriores)
output_filename = 'Datalog_merged_log_R.csv'
# ---------------------

# Lista que almacenará todas las filas para el CSV final / List to hold all rows for the final CSV
processed_rows = []

try:
    # with open(ruta, 'r') abre archivo en modo lectura y lo cierra automáticamente al salir del bloque
    with open(input_filename, 'r') as f:
        # Recorre el archivo línea por línea (eficiente en memoria)
        for line in f:
            # --- Skip Header Line ---
            # line.strip() elimina espacios/saltos; si queda vacío o contiene el encabezado, se omite
            if not line.strip() or 'Inicio de registro' in line:
                continue

            # --- Data Extraction ---
            # str.split(',') separa por comas → lista de campos
            parts = line.strip().split(',')

            # --- THE FIX: Check for malformed lines ---
            # Verifica que haya al menos dos campos (Date, Time). Si no, descarta.
            if len(parts) < 2:
                print(f"Skipping malformed line: {line.strip()}")
                continue
            # -----------------------------------------

            # Toma fecha y hora de las dos primeras columnas
            date_str = parts[0]
            time_str = parts[1]

            # Diccionario base de la fila
            row_dict = {
                'Date': date_str,
                'Time': time_str,
                'Unit': 'C°'  # Unidad fija para las lecturas
            }

            # --- Sensor Reading Extraction ---
            # ','.join(parts[2:]) recompone el resto de columnas para buscar "S#: valor"
            sensor_data_string = ','.join(parts[2:])

            # re.findall(patrón, texto) → lista de tuplas (clave, valor) que calzan el patrón
            # (S\d+) captura claves tipo S1, S2, ...
            # (-?\d+\.?\d*) captura números: signo opcional, enteros o decimales
            sensor_readings = re.findall(r'(S\d+):\s*(-?\d+\.?\d*)', sensor_data_string)

            # Convierte cada lectura a float y la añade al diccionario de la fila
            for key, value in sensor_readings:
                # float(cadena) convierte texto numérico a núm


SyntaxError: incomplete input (2858120046.py, line 61)

In [7]:
# Celda para crear Datalog_merged_log_R.csv (desde merged_testR.csv con columnas S1..S19)
import pandas as pd

# --- User settings ---
input_filename  = 'merged_testR.csv'
output_filename = 'Datalog_merged_log_R.csv'
# ---------------------

# Lee el CSV ya tabular
df = pd.read_csv(input_filename)

# Normaliza nombres (por si vienen con espacios)
df.columns = [c.strip() for c in df.columns]

# Verifica columnas esperadas
required = {'Date', 'Time'}
if not required.issubset(df.columns):
    raise ValueError(f"Faltan columnas requeridas {required} en {input_filename}")

# Detecta columna de unidad
unit_col = 'Unidad' if 'Unidad' in df.columns else ('Unit' if 'Unit' in df.columns else None)

# Lista de sensores esperados (S1..S19)
sensors = [f"S{i}" for i in range(1, 20)]
# Toma sólo los sensores que existan (por seguridad)
present_sensors = [c for c in sensors if c in df.columns]

# Convierte sensores a numérico (por si hay strings o vacíos)
for c in present_sensors:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# Arma DataFrame final
final_df = df[['Date', 'Time']].copy()
final_df['Unit'] = df[unit_col] if unit_col else 'C°'

# Añade sensores en orden
for c in sensors:
    if c in df.columns:
        final_df[c] = df[c]
    else:
        final_df[c] = pd.NA  # por si faltara alguno

# Exporta
final_df.to_csv(output_filename, index=False)
print(f"OK: {len(final_df)} filas guardadas en '{output_filename}'. Columnas: {list(final_df.columns)}")


OK: 6950 filas guardadas en 'Datalog_merged_log_R.csv'. Columnas: ['Date', 'Time', 'Unit', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19']
